In [118]:
%load_ext autoreload
%autoreload 2


import sys
import os

from sklearn.linear_model import SGDClassifier, LinearRegression, Lasso, Ridge
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
import seaborn as sn
import random
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from sklearn.manifold import TSNE
import tqdm
import copy
from sklearn.svm import LinearSVC 

from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import TruncatedSVD
import torch
from sklearn.linear_model import SGDClassifier
from rlace import solve_adv_game

from sklearn.svm import LinearSVC

import sklearn
from sklearn.linear_model import LogisticRegression
import random
import pickle
import matplotlib.pyplot as plt
from sklearn import cluster
from sklearn import neural_network
from gensim.models.keyedvectors import Word2VecKeyedVectors
from gensim.models import KeyedVectors
import numpy as np
import warnings
import argparse
from sklearn.neural_network import MLPClassifier
from collections import defaultdict
import scipy
from scipy import stats
from scipy.stats import pearsonr
import pandas as pd
from rlace import init_classifier
from collections import Counter
from sklearn.linear_model import LogisticRegression

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
bios_data_path = "../WWW/rlace/bios/data/bios_data"
bios_encodings_path = "../WWW/rlace/bios/data/encodings/freezed"

data = []
for mode in ["train", "dev", "test"]:
    with open(bios_data_path + "/" + "{}.pickle".format(mode), "rb") as f:
        data.append(pickle.load(f))
        
train,dev,test = data
gender2ind = {"m": 0, "f": 1}
train_profs, dev_profs = np.array([d["p"] for d in train]), np.array([d["p"] for d in dev])
train_gender, dev_gender = np.array([gender2ind[d["g"]] for d in train]), np.array([gender2ind[d["g"]] for d in dev])


data = []
for mode in ["train", "dev", "test"]:
    with open(bios_encodings_path + "/" + "{}_cls.npy".format(mode), "rb") as f:
        data.append(np.load(f))
        
train_x, dev_x, test_x = data
pca = PCA(n_components=256, random_state=0)
pca.fit(train_x)
train_x = pca.transform(train_x)
dev_x = pca.transform(dev_x)
test_x = pca.transform(test_x)

In [36]:
def get_probs(clf, x_train_projected, train_gender_binary):
    preds = clf.predict(x_train_projected)
    preds_male = clf.predict(x_train_projected[train_gender_binary==0])
    preds_female = clf.predict(x_train_projected[train_gender_binary==1])
    couner_preds, counter_preds_male, counter_preds_female = Counter(preds), Counter(preds_male),  Counter(preds_female)
    probs_all = {k:v/sum(couner_preds.values()) for k,v in couner_preds.items()}
    probs_male = {k:v/sum(counter_preds_male.values()) for k,v in counter_preds_male.items()}
    probs_female = {k:v/sum(counter_preds_female.values()) for k,v in counter_preds_female.items()}
    print(probs_all)
    print(probs_male)
    print(probs_female)
    return

    for k in probs_all.keys():
        if k not in probs_male:
            probs_male[k] = 1e-6
        
    preds_diffs = {k:abs(probs_all[k]-probs_male[k]) for k in probs_all.keys()}
    print(preds_diffs)
    
for (prof1, prof2) in [("dentist", "physician")]: #[("dentist", "physician"), ("psychologist", "professor"),  ("nurse", "physician"), ("professor", "attorney"), ("journalist", "attorney"), ("physician", "professor"), ("nurse", "attorney"), ("dentist", "journalist")]:
    for frac in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
        print("Now examinaing", prof1, prof2, "with frac=", frac)
        
        with open("projs-balanced-binary-pca=128/{}-{}-prop={}.pickle".format(prof1, prof2, frac), "rb") as f:
            data = pickle.load(f)
            x_train_binary, train_profs_binary, train_gender_binary = data["train"]["x"], data["train"]["profs"], data["train"]["gender"]
            x_dev_binary, dev_profs_binary, dev_gender_binary = data["dev"]["x"], data["dev"]["profs"], data["dev"]["gender"]
            output = data["output"]
            P = output["P"]
            x_train_projected = x_train_binary@P
            clf = init_classifier()
            clf.fit(x_train_projected, train_profs_binary)
            get_probs(clf, x_train_projected, train_gender_binary)
            
            clf_gender = init_classifier()
            clf_gender.fit(x_train_projected, train_gender_binary)
            print("gender score", clf_gender.score(x_train_projected, train_gender_binary))
            
            print("------------------------")
            
            clf = init_classifier()
            clf.fit(x_train_binary, train_profs_binary)
            get_probs(clf, x_train_binary, train_gender_binary)
            
            clf_gender = init_classifier()
            clf_gender.fit(x_train_projected, train_gender_binary)
            print("gender score", clf_gender.score(x_train_projected, train_gender_binary))

            clf_gender = init_classifier()
            clf_gender.fit(x_train_projected[train_profs_binary==prof1], train_gender_binary[train_profs_binary==prof1])
            print("gender score, prof1:", clf_gender.score(x_train_projected[train_profs_binary==prof1], train_gender_binary[train_profs_binary==prof1]))
            
            clf_gender = init_classifier()
            clf_gender.fit(x_train_projected[train_profs_binary==prof2], train_gender_binary[train_profs_binary==prof2])
            print("gender score, prof2:", clf_gender.score(x_train_projected[train_profs_binary==prof2], train_gender_binary[train_profs_binary==prof2]))
            
            print("=========================")
            #preds = clf.predict(x_train_projected)
            #preds_male = clf.predict(x_train_projected[train_gender_binary]==0)

Now examinaing dentist physician with frac= 0.1
{'physician': 0.49725860493451113, 'dentist': 0.5027413950654889}
{'physician': 0.3646055437100213, 'dentist': 0.6353944562899787}
{'physician': 0.629911666159001, 'dentist': 0.3700883338409991}
gender score 0.442583003350594
------------------------
{'dentist': 0.5012183978068839, 'physician': 0.49878160219311607}
{'dentist': 0.906183368869936, 'physician': 0.09381663113006397}
{'physician': 0.9037465732561681, 'dentist': 0.09625342674383186}
gender score 0.5217788607980506
gender score, prof1: 0.9984770027413951
gender score, prof2: 0.999695400548279
Now examinaing dentist physician with frac= 0.2
{'dentist': 0.49390801096558024, 'physician': 0.5060919890344198}
{'dentist': 0.6494060310691441, 'physician': 0.3505939689308559}
{'physician': 0.6615900091379836, 'dentist': 0.33840999086201645}
gender score 0.47045385318306426
------------------------
{'dentist': 0.501827596710326, 'physician': 0.4981724032896741}
{'dentist': 0.805665549802

In [100]:
output

{'score': 0.5001522997258605,
 'P_before_svd': array([[ 2.95993030e-01, -3.07224870e-01,  3.36853676e-02, ...,
          4.47473954e-04,  1.46941550e-03,  5.03013609e-03],
        [-3.07224870e-01,  8.65928710e-01,  1.47001147e-02, ...,
          1.95275235e-04,  6.41245046e-04,  2.19512428e-03],
        [ 3.36853676e-02,  1.47001147e-02,  9.98388220e-01, ...,
         -2.14107604e-05, -7.03086844e-05, -2.40682260e-04],
        ...,
        [ 4.47473954e-04,  1.95275235e-04, -2.14107604e-05, ...,
          9.99999716e-01, -9.33975343e-07, -3.19720539e-06],
        [ 1.46941550e-03,  6.41245046e-04, -7.03086844e-05, ...,
         -9.33975343e-07,  9.99996933e-01, -1.04989867e-05],
        [ 5.03013609e-03,  2.19512428e-03, -2.40682260e-04, ...,
         -3.19720539e-06, -1.04989867e-05,  9.99964060e-01]]),
 'P': array([[ 2.95993030e-01, -3.07224870e-01,  3.36853713e-02, ...,
          4.47473984e-04,  1.46941561e-03,  5.03013609e-03],
        [-3.07224870e-01,  8.65928710e-01,  1.470011

In [157]:
for (prof1, prof2) in [("dentist", "physician")]: #[("dentist", "physician"), ("psychologist", "professor"),  ("nurse", "physician"), ("professor", "attorney"), ("journalist", "attorney"), ("physician", "professor"), ("nurse", "attorney"), ("dentist", "journalist")]:
    for frac in [0.1]:
        with open("projs-balanced-binary-pca=128/{}-{}-prop={}.pickle".format(prof1, prof2, frac), "rb") as f:
            data = pickle.load(f)
            x_train_binary, train_profs_binary, train_gender_binary = data["train"]["x"], data["train"]["profs"], data["train"]["gender"]
            x_dev_binary, dev_profs_binary, dev_gender_binary = data["dev"]["x"], data["dev"]["profs"], data["dev"]["gender"]
            output = data["output"]
            P = output["P"]
            x_train_projected = x_train_binary@P
            
            clf =  LogisticRegression(max_iter=50000,penalty="l2",solver="saga",C=10.0, tol=1e-5) #init_classifier()
            clf.fit(x_train_projected, train_profs_binary)
            get_probs(clf, x_train_projected, train_gender_binary)
            print("main task accuracy", clf.score(x_train_projected, train_profs_binary))
            
            
            for i in range(3):
                gender_clf = LogisticRegression(max_iter=50000,penalty="l2",solver="saga",C=10.0, tol=1e-5) #SGDClassifier(loss="log",max_iter=500000, n_iter_no_change=10, tol=1e-6) #init_classifier()
                gender_clf.fit(x_train_projected, train_gender_binary)
                print("gender score", gender_clf.score(x_train_projected, train_gender_binary))

{'physician': 0.5079195857447456, 'dentist': 0.49208041425525434}
{'physician': 0.36826073713067314, 'dentist': 0.6317392628693268}
{'physician': 0.6475784343588181, 'dentist': 0.3524215656411818}
main task accuracy 0.7392628693268352
gender score 0.5204081632653061
gender score 0.5204081632653061
gender score 0.5204081632653061


In [202]:
def my_gender_classifier(x, clf):
    y_hat = clf.predict([x])[0]
    if y_hat == "dentist":
        return 0
    else:
        return 1

In [204]:
preds_gender = np.array([my_gender_classifier(x, clf) for x in x_train_projected])
(preds_gender==train_gender_binary).mean()

0.6396588486140725

In [213]:
predictor = torch.nn.Linear(128, 1)
bce_loss_fn = torch.nn.BCEWithLogitsLoss()
X_torch = torch.tensor(x_train_projected).float()
y_torch = torch.tensor(train_gender_binary).float()
optimizer = torch.optim.SGD(predictor.parameters(), lr=1e-4, momentum=0.9)
batch_size = 16

for i in range(25000):
    optimizer.zero_grad()
    idx = np.arange(0, X_torch.shape[0])
    np.random.shuffle(idx)
    X_batch, y_batch = X_torch[idx[:batch_size]], y_torch[idx[:batch_size]]
    bce = bce_loss_fn(predictor(X_batch).squeeze(), y_batch)
    bce.backward()
    optimizer.step()

In [214]:
preds = []
with torch.no_grad():
    preds = predictor(X_torch).squeeze()
preds = preds.detach().cpu().numpy()
preds = np.array([1 if p > 0 else 0 for p in preds])
acc = (preds==train_gender_binary).mean()
print(acc)

0.4968017057569296


In [177]:
0.6085897045385318-0.32988120621

0.2787084983285318

In [178]:
preds

array([0, 1, 1, ..., 0, 0, 0])

In [179]:
train_gender_binary

array([0, 1, 1, ..., 0, 0, 0])

In [180]:
train_gender_binary.shape, preds.shape

((6566,), (6566,))